In [3]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install trl
!pip install evaluate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
from transformers import pipeline
import torch
import pandas as pd
classifier = pipeline('sentiment-analysis')
print(classifier('We are very happy to show you the 🤗 Transformers library.'))
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from transformers import TrainingArguments
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer
from datasets import Dataset, DatasetDict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1614: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[{'label': 'POSITIVE', 'score': 0.9997795224189758}]


AttributeError: module 'pyarrow.lib' has no attribute 'ListViewType'

In [ ]:

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=3)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)



data = pd.read_csv('train.csv', sep='\t')
validation = pd.read_csv('validation.csv', sep='\t')
print(data)
print(data.shape)

mapping = {'positive': 2, 'neutral': 1, 'negative': 0}
data['label'] = data['label'].map(mapping)
print(data)
train_dataset = Dataset.from_pandas(data)
test_dataset = Dataset.from_pandas(validation)



dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})
print(dataset_dict)

In [ ]:
tokenized_datasets = dataset_dict.map(tokenize_function, batched=True)
print(tokenized_datasets)
# small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(15))
# small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(5))

In [ ]:
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments(output_dir="test_trainer"
                                  ,num_train_epochs=5
                                #   ,num_train_epochs=1
                                  , per_device_train_batch_size=8
                                  , per_device_eval_batch_size=8
                                  , warmup_steps=500
                                  , weight_decay=0.01
                                  ,learning_rate=2e-5)

training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch", push_to_hub=True)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= tokenized_datasets["train"],
    eval_dataset= tokenized_datasets["test"],
    # train_dataset=small_train_dataset,
    # eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:


# pt_model = AutoModelForSequenceClassification.from_pretrained("test_trainer", from_tf=True)
# pt_model.save_pretrained("test_trainer")


predict = trainer.predict(tokenized_datasets["test"])

print(confusion_matrix(predict.label_ids,np.argmax(predict.predictions, axis=-1),labels=[2, 1, 0]))
# report = classification_report(predict.label_ids, np.argmax(predict.predictions, axis=-1), target_names=[2, 1, 0])
# print(report)
model.save_pretrained("finbertSelfTrained")
tokenizer.save_pretrained("finbertSelfTrained")

# print

